In [ ]:
# 1. 라이브러리 및 환경 설정
from typing import TypedDict
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

C:\Users\Admin\AppData\Local\Temp\ipykernel_7072\407108958.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))


In [2]:
# 2. 상태 정의
class QAState(TypedDict):
    question: str
    route_label: str
    retrieved: str
    answer: str

In [3]:
# 3. 라우팅 판단 함수 (if문 기반)
def rule_based_router(state: QAState) -> QAState:
    question = state["question"]
    lowered = question.lower()

    if any(keyword in lowered for keyword in ["뉴스", "요즘", "오늘", "최근", "이슈"]):
        state["route_label"] = "web"
    elif any(keyword in lowered for keyword in ["회의", "요약", "기록", "회의록"]):
        state["route_label"] = "db"
    else:
        state["route_label"] = "general"

    print(f"🧠 [Rule 기반 판단] → {state['route_label']}")
    return state

# 조건 분기용 함수
def return_route_key(state: QAState) -> str:
    return state["route_label"]

In [ ]:
# 4. 분기별 노드 정의
def general_llm(state: QAState) -> QAState:
    response = llm.invoke(f"질문: {state['question']}\n\n위 질문에 답해주세요.")
    state["answer"] = response.content.strip()
    return state

def web_search(state: QAState) -> QAState:
    state["retrieved"] = "[웹 검색 결과] 오늘의 주요 뉴스를 요약했습니다."
    return state

def db_search(state: QAState) -> QAState:
    state["retrieved"] = "[DB 검색 결과] 회의 내용을 요약했습니다."
    return state

def generate_answer(state: QAState) -> QAState:
    if state.get("retrieved"):
        state["answer"] = f"[검색 기반 응답] {state['retrieved']}"
    return state

In [ ]:
# 5. LangGraph 구성 (rule 기반 판단 적용)
def build_rule_based_graph():
    builder = StateGraph(QAState)

    # 판단 노드만 rule 기반으로 적용
    builder.add_node("judge_route", RunnableLambda(rule_based_router))
    builder.add_node("general", RunnableLambda(general_llm))
    builder.add_node("web", RunnableLambda(web_search))
    builder.add_node("db", RunnableLambda(db_search))
    builder.add_node("generate", RunnableLambda(generate_answer))

    builder.set_entry_point("judge_route")

    builder.add_conditional_edges(
        "judge_route",
        return_route_key,
        {
            "general": "general",
            "web": "web",
            "db": "db"
        }
    )

    builder.add_edge("general", "generate")
    builder.add_edge("web", "generate")
    builder.add_edge("db", "generate")

    builder.set_finish_point("generate")

    return builder.compile()

In [6]:
# 6. 테스트 실행
graph = build_rule_based_graph()

questions = [
    "에이전트란 무엇인가요?",
    "오늘의 주요 뉴스 알려줘",
    "지난 회의 요약해줘"
]

for q in questions:
    print(f"\n❓ 질문: {q}")
    result = graph.invoke({
        "question": q,
        "route_label": "",
        "retrieved": "",
        "answer": ""
    })
    print(f"✅ 응답: {result['answer']}")


❓ 질문: 에이전트란 무엇인가요?
🧠 [Rule 기반 판단] → general
✅ 응답: "에이전트"라는 용어는 다양한 맥락에서 사용될 수 있지만, 일반적으로 다음과 같은 의미를 가집니다.

1. **일반적인 의미**: 에이전트는 어떤 일을 대신 수행하는 대리인이나 대행자를 의미합니다. 예를 들어, 부동산 에이전트는 고객을 대신하여 부동산 거래를 중개하는 역할을 합니다.

2. **컴퓨터 과학 및 인공지능**: 에이전트는 특정 환경에서 자율적으로 행동하고 결정을 내릴 수 있는 프로그램이나 시스템을 의미합니다. 예를 들어, 소프트웨어 에이전트는 사용자의 요구를 충족시키기 위해 정보를 수집하고 처리하는 역할을 합니다.

3. **비즈니스 및 마케팅**: 에이전트는 특정 제품이나 서비스를 홍보하고 판매하는 역할을 하는 사람이나 회사를 지칭할 수 있습니다. 이들은 제조업체와 소비자 간의 중개 역할을 합니다.

4. **법률**: 법률에서 에이전트는 다른 사람(주체)을 대신하여 법적 행위를 수행할 수 있는 권한을 가진 사람을 의미합니다.

따라서 "에이전트"라는 용어는 그 사용되는 분야에 따라 다양한 의미를 가질 수 있습니다.

❓ 질문: 오늘의 주요 뉴스 알려줘
🧠 [Rule 기반 판단] → web
✅ 응답: [검색 기반 응답] [웹 검색 결과] 오늘의 주요 뉴스를 요약했습니다.

❓ 질문: 지난 회의 요약해줘
🧠 [Rule 기반 판단] → db
✅ 응답: [검색 기반 응답] [DB 검색 결과] 회의 내용을 요약했습니다.
